<a href="https://colab.research.google.com/github/Yanina-Kutovaya/RecSys-retail/blob/main/notebooks/service_test_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Yanina-Kutovaya/RecSys-retail.git
!pip install -r RecSys-retail/requirements_Colab.txt

Cloning into 'RecSys-retail'...
remote: Enumerating objects: 1258, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 1258 (delta 67), reused 94 (delta 46), pack-reused 1132
Receiving objects: 100% (1258/1258), 308.79 KiB | 4.23 MiB/s, done.
Resolving deltas: 100% (669/669), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 334 kB/s 
     |████████████████████████████████| 72 kB 582 kB/s 
     |████████████████████████████████| 2.0 MB 42.5 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 
     |████████████████████████████████| 80 kB 6.4 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2

In [2]:
%cd RecSys-retail/service

/content/RecSys-retail/service


In [3]:
import logging
import sys
import os
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(),'..'))

In [4]:
import logging
import pandas as pd
from fastapi.testclient import TestClient

from main import app, Model
from src.recsys_retail.models.serialize import load
from src.recsys_retail.data.make_dataset import load_data
from src.recsys_retail.models.train import data_preprocessing_pipeline 
from scripts.train_save_model import train_store

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
logger = logging.getLogger(__name__)

client = TestClient(app)

MODEL = os.getenv("MODEL", default='LightGBM_v1')

In [7]:
def test_healthcheck():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json()["status"] == "Green"
    logger.info(f'status_code = 200, status = "Green"')

In [8]:
def test_predict():
    data, item_features, user_features = load_data()
    train_dataset_lvl_2 = data_preprocessing_pipeline(
        data, item_features, user_features
    )
    train_store(train_dataset_lvl_2, 'LightGBM_v1')
    Model.classifier = load(MODEL)

    user = {"user_id": 1340}
    response = client.post('/predict?user_id=1340', json=user)    
    assert response.status_code == 200
    assert response.json()[0]['user_id'] == 1340
    logger.info(f'test_predict single user status_code = 200, response: {response.json()}')    

    users = {"user_ids": [1340, 1364]}
    response = client.post('/predict_user_list?batch_id=1', json=users)
    assert response.status_code == 200
    assert response.json()[0]['user_ids'] == [1340, 1364]
    logger.info(f'test_predict user_list status_code = 200, response: {response.json()}')

In [9]:
test_healthcheck()
test_predict()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2501 [00:00<?, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2084, number of negative: 156236
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.149830 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20223
[LightGBM] [Info] Number of data points in the train set: 158320, number of used features: 131
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.013163 -> initscore=-4.317079
[LightGBM] [Info] Start training from score -4.317079
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[439]	training's auc: 0.913368	valid_1's auc: 0.842002
